In [ ]:
import os
import configparser
import pymongo
import pandas as pd

# Paths and directory
current_directory = os.getcwd()
config = configparser.ConfigParser()

config_file_path = os.path.join(current_directory, '03-configfiles', 'pw-config.ini')
config.read(config_file_path)

# Get the MongoDB connection string from the config file
mongo_client = config.get('mongo', 'connectiondb')
mongo_collection = config.get('mongo', 'collectionmongo')
mongo_db = config.get('mongo', 'dbmonog')

# Establish connection to MongoDB
client = pymongo.MongoClient(mongo_client)

# Select the database and collection
db = client[mongo_db]  
collection = db[mongo_collection]  # Replace 'your_collection_name' with your actual collection name

# Query data from MongoDB
data = collection.find({})

# Convert MongoDB documents to a list of dictionaries
data_list = list(data)  # Directly convert the cursor to a list

# Create a DataFrame
df = pd.DataFrame(data_list)

df

df.to_csv(current_directory+'/01-data/mongo/data_from_mongodb.csv', index=False)
